# Client

I've included the example image client here(you'll need the densenet_onnx from the Triton repo to run it) but hopefully this will serve as a solid baseline for building up our client structure.

In [1]:
!pip install tritonclient[all]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import triton_utils

from PIL import Image
import numpy as np
from attrdict import AttrDict

import tritonclient.grpc as grpcclient
import tritonclient.grpc.model_config_pb2 as mc
import tritonclient.http as httpclient
from tritonclient.utils import InferenceServerException
from tritonclient.utils import triton_to_np_dtype
import cv2

In [3]:
def read_frames(video_path, fidxs=None, grayscale=True):
    """Read frames from a video file.
    
    Args:
        video_path: Path to MP4
        fidxs: List of frame indices or None to read all frames (default: None)
        grayscale: Keep only one channel of the images (default: True)
    
    Returns:
        Loaded images in array of shape (n_frames, height, width, channels) and dtype uint8.
    """
    vr = cv2.VideoCapture(video_path)
    if fidxs is None:
        fidxs = np.arange(vr.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for fidx in fidxs:
        vr.set(cv2.CAP_PROP_POS_FRAMES, fidx)
        img = vr.read()[1]
        if grayscale:
            img = img[:, :, [0]]
        frames.append(img)
    return np.stack(frames, axis=0)

In [4]:
test_data = "../tensorrt/data/190719_090330_wt_18159206_rig1.2@15000-17560.mp4"
image_data = read_frames(test_data,np.arange(1))

In [5]:
from triton_utils import parse_model
from triton_utils import UserData
from triton_utils import requestGenerator

In [6]:
url = 'localhost:8001'
model_name = "bu_savedmodel"
model_version="1"
protocol = 'grpc'
triton_client = grpcclient.InferenceServerClient(
                url=url)

In [7]:
model_metadata = triton_client.get_model_metadata(model_name=model_name, model_version=model_version)

model_config = triton_client.get_model_config(model_name=model_name, model_version=model_version)

model_config = model_config.config

max_batch_size, input_name, output_name, c, h, w, format, dtype = parse_model(
        model_metadata, model_config)

In [8]:
requests = []
responses = []
result_filenames = []
request_ids = []
image_idx = 0
last_request = False
user_data = UserData()

sent_count = 0

In [10]:
batched_image_data = image_data
for inputs, outputs in requestGenerator(
        batched_image_data[0].astype('float32'), input_name, output_name, dtype, protocol=protocol):
    sent_count += 1
    print(outputs)
    responses.append(triton_client.infer(model_name,
                                inputs.astype('float32'),
                                request_id=str(sent_count),
                                model_version=model_version,
                                outputs=outputs))

TypeError: requestGenerator() got an unexpected keyword argument 'protocol'

In [ ]:
triton_client.infer(model_name,
                                inputs,
                                request_id=str(sent_count),
                                model_version=model_version,
                                outputs=outputs)